#### Turtle Trading

#### Data, Library Load

In [1]:
from tqdm import tqdm 
import pandas as pd
from copy import deepcopy
from dateutil.parser import parse
import datetime as dt
from pykrx import stock
import numpy as np

In [3]:
import sys
sys.path.append('C:/Users/mi-bi/OneDrive/바탕 화면/Pjt/Kis_Api_test/')
from BackTesting_System.backtesting import BackTesting,Utils

In [5]:
path = 'C:/Users/mi-bi/OneDrive/바탕 화면/data Row/mom.csv'
df = pd.read_csv(path,index_col=0)
df = df.drop(df.iloc[:,8:],axis=1)
df.head(1)

,날짜,종가,시가,최고가,최저가,누적거래량,누적거래대금,종목명
0,2022-12-22,10650,9980,11700,9930,28856232,309808795730,바이오노트


In [6]:
df.shape

(2621634, 8)

In [7]:
df = df[df.apply(lambda x: '스팩' not in x['종목명'], axis=1)]
# df = df[df.apply(lambda x: '대성홀딩스' not in x['종목명'], axis=1)]
df.shape

(2605276, 8)

#### function define

In [8]:
# make momentum (10일,20일)
def make_mom(df):
    df = deepcopy(df)
    buf = []
    for _, gdf in tqdm(df.groupby('종목명')):
        gdf['10_mom'] = gdf['종가'] / gdf['종가'].shift(11)*100
        gdf['20_mom'] = gdf['종가'] / gdf['종가'].shift(21)*100
        buf.append(gdf)
    del df
    res = pd.concat(buf)
    return res 

In [9]:
# make tr
def make_tr(df):
    df = deepcopy(df)
    buf = []
    for _, gdf in tqdm(df.groupby('종목명')):
        gdf['con1'] = gdf['최고가'] - gdf['종가'].shift(1)
        gdf['con2'] = gdf['최저가'] - gdf['종가'].shift(1)
        gdf['con1_per'] = (gdf['최고가'] - gdf['종가'].shift(1))/gdf['종가'].shift(1)*100
        gdf['con2_per'] = (gdf['최저가'] - gdf['종가'].shift(1))/gdf['종가'].shift(1)*100
        buf.append(gdf) 
    del df
    res = pd.concat(buf)
    res['con3'] = res['최고가'] - res['최저가']
    res['con3_per'] = ((res['최고가']-res['최저가'])/res['최저가']*100).abs()
    res['con1'] = res['con1'].abs()
    res['con2'] = res['con2'].abs()
    res['con1_per'] = res['con1_per'].abs()
    res['con2_per'] = res['con2_per'].abs()
    res['TR'] = res[['con1','con2','con3']].max(axis=1)
    res['TR_per'] = res[['con1_per','con2_per','con3_per']].max(axis=1)
    res = res.drop(columns=['con1', 'con2', 'con3'], axis=1)
    res = res.drop(columns=['con1_per','con2_per','con3_per'], axis=1)
    return res 

In [10]:
# make atr 21일
def make_atr(df):
    df = deepcopy(df)
    buf = []
    for _, gdf in tqdm(df.groupby('종목명')):
        gdf['ATR'] = gdf['TR'].rolling(21).mean()
        gdf['ATR_per'] = gdf['TR_per'].rolling(21).mean()
        gdf['ATR'].fillna(np.inf, inplace=True)
        gdf['ATR_per'].fillna(np.inf, inplace=True)
        buf.append(gdf)
    del df
    res = pd.concat(buf)
    res = res.drop(columns=['TR'], axis=1)
    return res 

In [11]:
def make_raking(df):
    df = deepcopy(df)
    buf = []
    for _,gdf in tqdm(df.groupby('날짜')):
        gdf['ranking'] = gdf['ATR'].rank(method='min', ascending=True)
        buf.append(gdf)
    del df
    res = pd.concat(buf)
    return res

In [12]:
# make buy
def make_buy(df):
    df = deepcopy(df)
    buf = []
    for _, gdf in tqdm(df.groupby('종목명')):
        gdf['buy1'] = (gdf['20_mom']>100)&(gdf['종가'].rolling(window=20, closed='left').max() < gdf['종가'])  #&(gdf['ranking'] <= 10)     
        #gdf['middle_buy'] = (gdf['종가'].rolling(window=55, closed='left').max() < gdf['종가'])
        #gdf['buy'] = gdf['short_buy'] | gdf['middle_buy']
        buf.append(gdf)
    del df
    res = pd.concat(buf)
    df = deepcopy(res)
    del buf
    buf = []
    for (_, buy), gdf in tqdm(df.groupby(['날짜', 'buy1'])):
        if buy == True:
            gdf['buy2'] = gdf['ATR_per'].rank(method='min', ascending=True) <= 10
        buf.append(gdf)
    res = pd.concat(buf)
    res['buy'] = res['buy1'] & res['buy2']
    res = res.drop(columns=['buy1', 'buy2'], axis=1)
    return res

In [13]:
# make sell
def make_sell(df):
    df = deepcopy(df)
    buf = []
    for _,gdf in tqdm(df.groupby('종목명')):
        con1 = (gdf['10_mom'] > 100)&(gdf['종가'].rolling(window=10,closed='left').min() > gdf['종가'])
        #con2 = (gdf['10_mom'] < 100)&(gdf['종가'].rolling(window=10,closed='left').max() < gdf['종가'])
        gdf['sell'] = con1
        buf.append(gdf)
    del df
    res = pd.concat(buf)

    return res 

#### Do!

In [14]:
df = make_mom(df)

100%|██████████| 2362/2362 [00:02<00:00, 825.19it/s]


In [16]:
df = make_tr(df)

100%|██████████| 2362/2362 [00:06<00:00, 380.46it/s]


In [17]:
df = make_atr(df)

100%|██████████| 2362/2362 [00:03<00:00, 706.12it/s]


In [15]:
# df = make_raking(df)

In [18]:
df = make_buy(df)

100%|██████████| 2405/2405 [00:01<00:00, 1817.16it/s]


In [19]:
df = make_sell(df)

100%|██████████| 2362/2362 [00:02<00:00, 1093.93it/s]


In [20]:
code_kospi = pd.read_csv('../../SRC/current_kospi_codes.csv')
code_kosdaq = pd.read_csv('../../SRC/current_kosdaq_codes.csv')
code = pd.concat([code_kospi,code_kosdaq])
code.rename(columns={'회사명':'종목명'},inplace=True)
code['종목코드'] = code['종목코드'].astype('str')

df_signal = df.merge(code,on='종목명',how='inner')
df_signal.rename(columns={'종목코드':'코드'},inplace=True)
df_signal['구매수량']=1

In [21]:
df_signal['코드'] = df_signal['코드'].apply(lambda x: str(x).zfill(6))

In [22]:
bt = BackTesting()
ut = Utils()

### 5년치 한꺼번에

In [37]:
first_date = parse(sorted(df_signal['날짜'].unique())[0])
last_date = parse(sorted(df_signal['날짜'].unique())[-1])
df_signal_test = df_signal
df_stock = stock.get_index_fundamental(str(first_date)[:10], str(last_date)[:10], '1001')
bt.MONEY = 10000000
first_monry = bt.MONEY
bt.buy_sell(df=df_signal_test,buy='buy',sell='sell', use_slippage=True)
bt.update_information()


a = pd.DataFrame(bt.ASSET).T
a['코드'] = a.index
b = df_signal[df_signal['날짜'] == sorted(df_signal_test['날짜'].unique())[-1]][['코드', '종가']]
df_join = a.merge(b,on='코드', how='inner')
res = pd.DataFrame((df_join['종가'] - df_join['평단가'])  * df_join['수량']).sum()[0]
c = df_join[['종가']].apply(lambda x: bt.calc_tax(x['종가'], '매도', True), axis=1).sum()

print('-'*50)
print('기간:' + str(first_date) + '~' + str(last_date))
print('승률:', round(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0] / bt.ECONO_INFORM.shape[0]*100,2),'%')
print('승리횟수:' + str(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0]))
print('전체횟수:' + str(bt.ECONO_INFORM.shape[0]))
print('평가손익:', round(res, 2) , ' ', '평가손익율:', round(res / (df_join['평단가']  * df_join['수량']).sum() * 100, 2))
print('실현손익:', bt.ECONO_INFORM['실현손익'].sum())
print('최대손실폭(실현손익):',bt.ECONO_INFORM['실현손익'].cumsum().min())
print('최대수익폭(실현손익):',bt.ECONO_INFORM['실현손익'].cumsum().max())
print('누적수익률', round(((bt.ECONO_INFORM['실현손익'].sum() + res - (bt.JOUNAL[bt.JOUNAL['매매구분']=='매수'][['매매비용']].sum()) - c).iloc[0])/ first_monry * 100, 2),'%')
print('시장수익률:', + round(((df_stock.iloc[-1, 0] - df_stock.iloc[0, 0]) / df_stock.iloc[0, 0]) *100, 2), '%')
print('CAGR:', round((((bt.MONEY + (df_join['종가'] * df_join['수량']).sum() - c) / first_monry) ** (1/5) - 1)*100, 2), '%')
print('-'*50)

100%|██████████| 2405/2405 [02:15<00:00, 17.69it/s]


--------------------------------------------------
기간:2018-01-29 00:00:00~2022-12-29 00:00:00
승률: 20.27 %
승리횟수:236
전체횟수:1164
평가손익: 1582456.0   평가손익율: 8.32
실현손익: 12444043.0
최대손실폭(실현손익): -578032.9999999994
최대수익폭(실현손익): 13064511.0
누적수익률 139.61 %
시장수익률: -13.92 %
CAGR: 19.1 %
--------------------------------------------------


In [39]:
print('전략의 손익비 최소 8.25 이상')

전략의 손익비 최소 8.25 이상


In [24]:
# 마지막 유동성 + 계좌평가금액 - 계좌 청산시 비용
bt.MONEY + (df_join['종가']*df_join['수량']).sum() - c

23961451.0

In [25]:
# 처음 유동성 + 실현손익 + 평가손익 - 총매수비용 - 계좌 청산시 비용
first_monry + bt.ECONO_INFORM['실현손익'].sum() + res - (bt.JOUNAL[bt.JOUNAL['매매구분']=='매수'][['매매비용']].sum()) - c

매매비용    23961451.0
dtype: object

In [42]:
bt.JOUNAL['종목코드']

0       102260
1       094800
2       001810
3       002840
4       033560
         ...  
8932    078070
8933    041830
8934    044490
8935    088350
8936    000540
Name: 종목코드, Length: 8937, dtype: object

In [57]:
bt.JOUNAL

,날짜,종목코드,체결단가,체결수량,매매비용,매매구분
0,2018-03-02,102260,6780,51,61.0,매수
1,2018-03-02,094800,3785,352,236.0,매수
2,2018-03-02,001810,3340,120,71.0,매수
3,2018-03-02,002840,35531,10,63.0,매수
4,2018-03-02,033560,7570,47,63.0,매수
...,...,...,...,...,...,...
8932,2022-12-29,078070,16450,14,41.0,매수
8933,2022-12-29,041830,20200,9,32.0,매수
8934,2022-12-29,044490,9800,18,31.0,매수
8935,2022-12-29,088350,2770,68,33.0,매수


In [56]:
ut.making_mt_plot(handle_data=df_signal_test, code='033560', jounal=bt.JOUNAL)

    

2018-03-02 2018-03-07
2021-11-29 2022-07-04


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'close': array([7270, 7150, 7100, ..., 3855, 3785, 3700], dtype=int64),
              'high': array([7350, 7280, 7150, ..., 3860, 3880, 3750], dtype=int64),
              'low': array([7220, 7090, 7010, ..., 3690, 3710, 3560], dtype=int64),
              'open': array([7270, 7280, 7120, ..., 3690, 3815, 3750], dtype=int64),
              'type': 'candlestick',
              'x': array(['2018-01-29', '2018-01-30', '2018-01-31', ..., '2022-12-27',
                          '2022-12-28', '2022-12-29'], dtype=object)}],
    'layout': {'height': 500,
               'shapes': [{'fillcolor': 'gray',
                           'line': {'width': 0},
                           'opacity': 0.25,
                           'type': 'rect',
                           'x0': '2018-03-02',
                           'x1': '2018-03-07',
                           'xref': 'x',
                           'y0': 0,
                           'y1': 1,
                           'yref': 'y domain'},
                          {'fillcolor': 'gray',
                           'line': {'width': 0},
                           'opacity': 0.25,
                           'type': 'rect',
                           'x0': '2021-11-29',
                           'x1': '2022-07-04',
                           'xref': 'x',
                           'y0': 0,
                           'y1': 1,
                           'yref': 'y domain'}],
               'template': '...',
               'title': {'text': "종목명: ['블루콤']  |  코드: 033560"},
               'width': 1000,
               'xaxis': {'categoryorder': 'category ascending', 'nticks': 5, 'type': 'category'}}
})

In [47]:
import matplotlib.pyplot as plt